In [1]:
import numpy as np
from cmocean import cm
import cartopy as cp
import cartopy.crs as ccrs
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr
import sys
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE')
#list of models
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/UTILS')
import lom
import utils as ut

%matplotlib inline
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import cartopy.feature as cfeature
from importlib import reload
import matplotlib.path as mpath
import glob
import pickle
import pandas as pd
import seawater
import time
plt.rcParams.update({'font.size': 12})
font = {'family' : 'normal',
'weight' : 'normal',
'size'   : 12}

plt.rc('font', **font)

## problem description:

- When downloading and processing ERA5 forcing, occasionally, the variable called `bulk_*_9_era5_daily.nc` gives a time dimension of time 0 (instead of 365/366), breaking the run
- I wrote a script telling me when this problem happens: 





In [2]:
# 1940 1949 1953 1958 1959 1960 1962 1963 1964 1968

yrs = [1940, 1949, 1953, 1958 , 1959, 1960 , 1962, 1963, 1964, 1968]

#/gpfs/data/greenocean/software/products/ERA5_v202303_TJ/PATCH_HOLE


pres.era.1968.daily.nc

In [7]:
tdir = '/gpfs/data/greenocean/software/products/ERA5_v202303_TJ/'
print('ok')
yr = 1961
t = xr.open_dataset(f'{tdir}/2m_dewpoint_temperature_ERA5_{yr}.nc')
print(t)

print('not ok')
yr = 1960
t = xr.open_dataset(f'{tdir}/2m_dewpoint_temperature_ERA5_{yr}.nc')
print(t)


print('no')
t = xr.open_dataset(f'{tdir}/pres.era.1960.daily.nc')
print(t)


print('yes')
t = xr.open_dataset(f'{tdir}/pres.era.1961.daily.nc')
print(t)

ok
<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 8760)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.25 359.5 359.75
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1961-01-01 ... 1961-12-31T23:00:00
Data variables:
    d2m        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-04-15 04:09:24 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...
not ok
<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 8784)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.25 359.5 359.75
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1960-01-01 ... 1960-12-31T23:00:00
Data variables:
    d2m        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-04-15 01:37:54 GMT by grib_to_netc